## EPC Extent

In [10]:
help(gdal.DEMProcessingOptions)

Help on function DEMProcessingOptions in module osgeo.gdal:

DEMProcessingOptions(options=None, colorFilename=None, format=None, creationOptions=None, computeEdges=False, alg='Horn', band=1, zFactor=None, scale=None, azimuth=None, altitude=None, combined=False, multiDirectional=False, igor=False, slopeFormat=None, trigonometric=False, zeroForFlat=False, addAlpha=None, callback=None, callback_data=None)
    Create a DEMProcessingOptions() object that can be passed to gdal.DEMProcessing()
    Keyword arguments are :
      options --- can be be an array of strings, a string or let empty and filled from other keywords.
      colorFilename --- (mandatory for "color-relief") name of file that contains palette definition for the "color-relief" processing.
      format --- output format ("GTiff", etc...)
      creationOptions --- list of creation options
      computeEdges --- whether to compute values at raster edges.
      alg --- 'ZevenbergenThorne' or 'Horn'
      band --- source band numb

In [6]:
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed

json_pipe_base = """ { "pipeline": [ %s ] } """

gdal_writer = """
    "%s",
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW"
    }
"""

resolution = 0.8/0.3048

ortho_folder = r"../EPCExtent_30cm/Orthos/"
laz_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full"
dsm_folder = f"../EPCExtent_30cm/Elevation_80cmNPS/DSM_{resolution}"
os.makedirs(dsm_folder, exist_ok=True)

infiles = []
for f in os.listdir(laz_folder):
    infile = os.path.join(laz_folder, f)
    outfile = os.path.join(dsm_folder, f.replace(".laz", f"_DSM{round(resolution*.3048*100,0)}cm.tif"))
    if f.endswith(".laz"):# and not os.path.exists(outfile):
        files = {}
        files["in_file"] = infile
        files["out_file"] = outfile
        infiles.append(files)
        
def createDSM(file_dict, res, overwrite=False):
    infile = file_dict["in_file"]
    outfile = file_dict["out_file"]
    
    if os.path.exists(outfile) and not overwrite:
        return
  
    json_dsm = gdal_writer % (infile.replace("\\","/"), outfile.replace("\\","/"), res, max(res * 2,10))#, ox, oy)#, raswidth, rasheight)
    json_dsm = json_pipe_base % json_dsm
    
    pipeline = pdal.Pipeline(json_dsm)
    pipeline.loglevel = 8

    start = datetime.now()
    pipeline.execute()
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {outfile} - {end-start} elapsed")
    

print("\nBeginning DSM Generations for {} QQuads\n".format(len(infiles)))
Parallel(n_jobs=5, max_nbytes=None, verbose=30)(delayed(createDSM)(f, resolution) for f in infiles)
for f in infiles:
    hour = int(datetime.strftime(datetime.now(), '%H'))
    #while hour > 9 and hour < 17:
    #    print("Pausing creation for 1 hours...")
    #    time.sleep(60*60)
    #    hour = int(datetime.strftime(datetime.now(), '%H'))
        
    createDSM(f)
    
print("\n\n...FINISHED\n\n")


Beginning DSM Generations for 327 QQuads



[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   4 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   5 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  11 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  12 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  13 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  14 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.9s
[Parallel(

TypeError: createDSM() missing 1 required positional argument: 'res'

## URBAN EXTENT

In [ ]:
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed
import time

json_pipe_base = """ { "pipeline": [ %s ] } """

gdal_writer = """
    "%s",
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW"
    }
"""

ortho_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Orthos"
laz_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\RGB\Full_retiled"
dsm_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\DSM_2ft"
os.makedirs(dsm_folder, exist_ok=True)

infiles = []
for f in os.listdir(laz_folder):
    infile = os.path.join(laz_folder, f)
    outfile = os.path.join(dsm_folder, f.replace(".laz", "_DSM.tif"))
    if f.endswith(".laz") and not os.path.exists(outfile):
        files = {}
        files["in_file"] = infile
        files["out_file"] = outfile
        infiles.append(files)
        
def createDSM(file_dict):
    infile = file_dict["in_file"]
    outfile = file_dict["out_file"]
    
    #if os.path.exists(outfile):
    #    return

    ortho_tif = os.path.join(ortho_folder, os.path.basename(infile).replace(".laz", ".tif"))
    with rio.open(ortho_tif) as ras:
        res = ras.res[0]
        raswidth = ras.width
        rasheight = ras.height
        ox = ras.transform[2]
        oy = ras.transform[5]


    #override resolution. Can't match ortho res
    #res = 0.984251969
    res = 2 # Should be able to support 1.3 ft, but set to 2 to match 2015 Elevation Data. 
    json_dsm = gdal_writer % (infile.replace("\\","/"), outfile.replace("\\","/"), res, res * 2)#, ox, oy)#, raswidth, rasheight)
    json_dsm = json_pipe_base % json_dsm
    
    pipeline = pdal.Pipeline(json_dsm)
    pipeline.loglevel = 8

    start = datetime.now()
    pipeline.execute()
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {outfile} - {end-start} elapsed")
    

print("\nBeginning DSM Generations for {} QQuads\n".format(len(infiles)))
#Parallel(n_jobs=1, max_nbytes=None, verbose=30, prefer="threads", temp_folder="./") \
#        (delayed(createDSM)(f) for f in infiles)
for f in infiles:
    #if "E0980_N510_1" in f['out_file']:
    hour = int(datetime.strftime(datetime.now(), '%H'))
    """while hour > 9 and hour < 17:
        print("Pausing creation for 1 hours...")
        time.sleep(60*60)
        hour = int(datetime.strftime(datetime.now(), '%H'))"""
        
    createDSM(f)
    
print("\n\n...FINISHED\n\n")


Beginning DSM Generations for 217 QQuads

2019-12-17 09:57:24.591319	-	Finished with P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\DSM_2ft\E0920_N450_3_DSM.tif - 0:01:15.189486 elapsed
